<a href="https://colab.research.google.com/github/hasune613/hello-world/blob/main/Pytorch_BERT_beginner's_room%E3%80%90%E6%97%A5%E6%9C%AC%E8%AA%9Eversion%E3%80%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
debug = True

In [ ]:
# debug2 = True

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

# import os
# import json
# f = open("/content/drive/My Drive/kaggle.json", 'r')
# json_data = json.load(f) 
# os.environ['KAGGLE_USERNAME'] = json_data['username']
# os.environ['KAGGLE_KEY'] = json_data['key']

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 68 bytes


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold

In [2]:
! pip -q install transformers


In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import matplotlib.pyplot as plt

import transformers
import random

import warnings
warnings.simplefilter('ignore')

scaler = torch.cuda.amp.GradScaler()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cpu')

In [4]:
SEED = 508

def random_seed(SEED):
    random.seed(SEED)
    os.environ['PYTHONHASHSEED' ] = str(SEED)
    np.random.seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
random_seed(SEED)

In [5]:
train = "/content/drive/MyDrive/kaggle/Pytorch BERT beginner's room【日本語version】/train.csv"

In [6]:
train = pd.read_csv(train)
train.head(3)

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676


In [7]:
train.shape

(2834, 6)

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2834 entries, 0 to 2833
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2834 non-null   object 
 1   url_legal       830 non-null    object 
 2   license         830 non-null    object 
 3   excerpt         2834 non-null   object 
 4   target          2834 non-null   float64
 5   standard_error  2834 non-null   float64
dtypes: float64(2), object(4)
memory usage: 133.0+ KB


In [9]:
train.columns

Index(['id', 'url_legal', 'license', 'excerpt', 'target', 'standard_error'], dtype='object')

In [10]:
test= "/content/drive/MyDrive/kaggle/Pytorch BERT beginner's room【日本語version】/test.csv"

In [11]:
test = pd.read_csv(test)
test.head(3)

,id,url_legal,license,excerpt
0,c0f722661,NaN,NaN,My hope lay in Jack's promise that he would ke...
1,f0953f0a5,NaN,NaN,Dotty continued to go to Mrs. Gray's every nig...
2,0df072751,NaN,NaN,It was a bright and cheerful scene that greete...


In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         7 non-null      object
 1   url_legal  3 non-null      object
 2   license    3 non-null      object
 3   excerpt    7 non-null      object
dtypes: object(4)
memory usage: 352.0+ bytes


In [13]:
sample = "/content/drive/MyDrive/kaggle/Pytorch BERT beginner's room【日本語version】/sample_submission.csv"

In [14]:
sample = pd.read_csv(sample)
sample.head()

,id,target
0,c0f722661,0.0
1,f0953f0a5,0.0
2,0df072751,0.0
3,04caf4e0c,0.0
4,0e63f8bea,0.0


In [15]:
train = train.sort_values(['target']).reset_index(drop=True)
train.head(3)

,id,url_legal,license,excerpt,target,standard_error
0,4626100d8,NaN,NaN,"The commutator is peculiar, consisting of only...",-3.676268,0.623621
1,493b80aa7,NaN,NaN,The Dunwich horror itself came between Lammas ...,-3.668360,0.571404
2,fe44cbd14,NaN,NaN,"The iron cylinder weighs 23 kilogrammes; but, ...",-3.642892,0.644398


In [16]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

In [17]:
train['kfold'] = train.index % 5
train.head(3)

,id,url_legal,license,excerpt,target,standard_error,kfold
0,4626100d8,NaN,NaN,"The commutator is peculiar, consisting of only...",-3.676268,0.623621,0
1,493b80aa7,NaN,NaN,The Dunwich horror itself came between Lammas ...,-3.668360,0.571404,1
2,fe44cbd14,NaN,NaN,"The iron cylinder weighs 23 kilogrammes; but, ...",-3.642892,0.644398,2


In [18]:
p_train = train[train['kfold']!=0].reset_index(drop=True)
p_valid = train[train['kfold']==0].reset_index(drop=True)

In [21]:
p_train.head()

,id,url_legal,license,excerpt,target,standard_error,kfold
0,493b80aa7,NaN,NaN,The Dunwich horror itself came between Lammas ...,-3.668360,0.571404,1
1,fe44cbd14,NaN,NaN,"The iron cylinder weighs 23 kilogrammes; but, ...",-3.642892,0.644398,2
2,284eaa5ad,NaN,NaN,As to surface-slope its measurement—from nearl...,-3.639936,0.603819,3
3,9e9eacb49,NaN,NaN,"The tree is dioecious, bearing male catkins on...",-3.636834,0.606822,4
4,ee7d40251,NaN,NaN,We have frequent inquiries as to the best mean...,-3.585369,0.588952,1


In [22]:
p_valid.head(3)

,id,url_legal,license,excerpt,target,standard_error,kfold
0,4626100d8,NaN,NaN,"The commutator is peculiar, consisting of only...",-3.676268,0.623621,0
1,466e33a64,NaN,NaN,The copper even of such a conductor has been m...,-3.596751,0.567050,0
2,99a602911,NaN,NaN,The soil most suitable for the full developmen...,-3.531687,0.595653,0


In [19]:
max_sens = 314

class BERTDataSet(Dataset):

    def __init__(self,sentences,targets):
        
        self.sentences = sentences
        self.targets = targets

    def __len__(self):
        
        return len(self.sentences)
    
    def __getitem__(self,idx):
        
        sentence = self.sentences[idx]
        
        bert_sens = tokenizer.encode_plus(
            sentence, 
            add_special_tokens = True,
            max_length = max_sens,
            pad_to_max_length = True,
            return_attention_mask = True
        )
        ids = torch.tensor(bert_sens['input_ids'],dtype=torch.long)
        mask = torch.tensor(bert_sens['attention_mask'],dtype=torch.long)
        token_type_ids = torch.tensor(bert_sens['token_type_ids'],dtype=torch.long)
        target = torch.tensor(self.targets[idx],dtype=torch.float)

        return {
                'ids': ids,
                'mask': mask,
                'token_type_ids': token_type_ids,
                'targets': target
                }


In [20]:
train_dataset = BERTDataSet(p_train['excerpt'],p_train['target'])
valid_dataset = BERTDataSet(p_valid['excerpt'],p_valid['target'])

In [25]:
train_dataset[0]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{'ids': tensor([  101,  1996, 24654, 12414,  5469,  2993,  2234,  2090, 16983,  9335,
          1998,  1996,  1041, 12519, 11636,  1999,  4662,  1010,  1998,  2852,
          1012,  2849,  6590,  3351,  2001,  2426,  2216,  2040,  9741,  2049,
         21668, 18877,  1012,  2002,  2018,  2657,  1010,  5564,  1010,  1997,
          2054, 22352,  1005,  1055, 27707,  4440,  2000,  4729,  1010,  1998,
          1997,  2010, 15762,  4073,  2000, 17781,  2030,  6100,  2013,  1996,
         26785,  4948, 22026,  2239,  2012,  1996, 21255,  2121,  3075,  1012,
          2216,  4073,  2018,  2042,  1999, 15784,  1010,  2144,  2849,  6590,
          3351,  2018,  3843, 16234,  1997,  1996, 10326,  4355,  8015,  2000,
          2035, 13850,  2015,  2383,  3715,  1997,  1996, 14436,  2098,  3872,
          1012, 26151,  2018,  2042, 16880,  2135,  6091,  2012,  4729,  1025,
         11480,  2005,  1996,  2338,  1010,  2664,  2471,  8053, 11480,  2000,
          2131,  2188,  2153,  1010,  2004,  

In [21]:
train_batch = 16
valid_batch = 32

In [22]:
train_dataloader = DataLoader(train_dataset,batch_size = train_batch, shuffle=True, num_workers=8, pin_memory=True)
valid_dataloader= DataLoader(valid_dataset, batch_size=valid_batch, shuffle=False, num_workers=8, pin_memory=True)

In [ ]:
for i in train_dataloader:
    print(i)
    break

{'ids': tensor([[ 101, 2055, 2702,  ...,    0,    0,    0],
        [ 101, 2004, 2002,  ...,    0,    0,    0],
        [ 101, 2000, 3965,  ...,    0,    0,    0],
        ...,
        [ 101, 2071, 2017,  ...,    0,    0,    0],
        [ 101, 1996, 4971,  ...,    0,    0,    0],
        [ 101, 1000, 2092,  ...,    0,    0,    0]]), 'mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'targets': tensor([-0.7014, -1.0551, -2.1773, -2.3114, -1.6915, -0.4535, -1.4889, -0.2752,
        -0.2868, -1.1573, -0.5728,  0.3260, -1.4134,  0.7797, -2.8853, -0.9596])}


In [33]:
model = transformers.BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [34]:
model.to(device)

model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [29]:
for a in train_dataloader:
    ids = a['ids'].to(device)
    mask = a['mask'].to(device)
    # tokentype = a['token_type_ids].to(device)

    output = model(ids, mask)
    break


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

In [30]:
output['logits'].shape

torch.Size([16, 1])

In [31]:
output['logits'].squeeze(-1)

tensor([ 0.2165, -0.3243, -0.3914, -0.1725, -0.4280, -0.3088, -0.3065, -0.0460,
        -0.4011, -0.1498, -0.4898, -0.4790, -0.3191, -0.2400, -0.3383, -0.4310],
       grad_fn=<SqueezeBackward1>)

In [32]:
output = output['logits'].squeeze(-1).shape
output

torch.Size([16])

In [35]:
from transformers import AdamW
LR = 2e-5
optimizer =AdamW(model.parameters(), LR, betas=(0.9, 0.999),weight_decay=1e-2)

In [25]:
from transformers import get_linear_schedule_with_warmup

In [36]:
epochs = 1

train_steps = int(len(p_train) / train_batch * epochs)
print(train_steps)
num_steps = int(train_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer,num_steps,train_steps)

141


In [33]:
# RMSE
def loss_fn(output, target):
    return torch.sqrt(nn.MSELoss()(output, target))

In [34]:
def training(train_dataloader, model, optimizer , scheduler):
    model.train()
    torch.backends.cudnn.banchmark = True

    allpreds = []
    alltargets = []

    for a in train_dataloader:
        losses = []
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            ids = a['ids'].to(device, non_blocking=True)
            mask = a['mask'].to(device, non_blocking=True)
            tokentype = a['token_type_ids'].to(device, non_blocking=True)

            output = model(ids, mask)
            output = output['logits'].squeeze(-1)
            target = a['targets'].to(device, non_blocking=True)
            loss = loss_fn(output, target)

            losses.append(loss.item())
            allpreds.append(output.detach().cpu().numpy())
            alltargets.append(target.detach().squeeze(-1).cpu().numpy())

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        del loss

        scheduler.step()

    allpreds = np.concatenate(allpreds)
    alltargets = np.concatenate(alltargets)

    losses = np.mean(losses)

    train_rme_loss = np.sqrt(mean_squared_error(allpreds, altargets))
    return loss, train_rme_loss

In [26]:
train_dataloader = DataLoader(train_dataset, batch_size=train_batch, shuffle=True, num_workers=4, pin_memory=True)

scheduler = get_linear_schedule_with_warmup(optimizer, num_steps, train_steps)

losses, train_rme_loss = training(train_dataloader,model, optimizer,scheduler)
print(loss, train_rme_loss)

In [37]:
def validating(valid_dataloader, model):
    model.eval()
    allpareds = []
    alltargets = []

    for a in valid_dataloader:
        losses = []

        with torch.no_grad():
            ids = a['ids'].to(device)
            mask = a['mask'].to(device)
            tokentype = a['token_type_ids'].to(device)

            output = model(ids, mask)
            output = output['logits'].squeeze(-1)
            target = a['targets'].to(device)
            loss = loss_fn(output, target)

            losses.append(loss.item())
            allpreds.append(output.detach().cpu().numpy())
            alltargets.append(output.detach().cpu().numpy())

            del loss

        np.concatenate(allpreds)
        np.concatenate(alltargets)

        losses = np.mean(losses)

        valid_rme_loss = np.sqrt(mean_squared_error(alltargets, allpreds))

        return allpreds, losses, valid_rme_loss



In [27]:
allpreds, losses, valid_rme_loss = validating(valid_dataloader, model)
print(allpreds)
print(losses)
print(valid_rme_liss)

In [ ]:
# initializing the data
p_train = train[train['kfold'] != 0].reset_index(drop=True)
p_valid = train[train['kfold'] == 0].reset_index(drop=True)

train_dataset = BERTDataSet(p_train['excerpt'],p_train['target'])
valid_dataset = BERTDataSet(p_valid['excerpt'],p_train['target'])

train_dataloader = DataLoader(train_dataset, batch_size = train_batch, shuffle=True, num_workers=4, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=valid_batch, shuffle=False, num_workers=4, pin_memory=True)

model = transformers.BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=1)

model.to(device)
LR = 2e-5
optimizer = AdamW(model.parameters(), LR, betas=(0.9, 0.999),weight_decay=1e-2)

train_steps = int(len(p_train) / train_batch * epochs )

num_steps = int(train_steps * 0.1)

scheduler = get_linear_schedule_with_warmup(optimizer, num_steps, train_steps)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [28]:
trainlosses = []
vallosses = []
bestscore = None

trainscores = []
validscores = []

for epoch in tqdm(range(epochs)):

    print(' = *10' + str(epoch + 1) + 'start=*10')

    trainloss, trainscore = training(train_dataloader, model, optimizer, scheduler)
    trainlosses.append(trainloss)
    trainscores.append(trainscore)

    print('trainscore is ' + str(trainscore))

    preds, validloss, valscore = validating(valid_dataloader, model)

    vallosses.append(validloss)
    validscores.append(valscore)

    print('valscore is' + str(valscore))

    if bestscore is None:
        bestscore = valscore
        print('Save first model')

        state = {
            'state_dict' : model.state_dict(),
            'optimizer_dict' : optimizer.state_dict(),
            'bestscore' : bestscore
        }
        torch.save(state, 'mode10.pth')

    elif bestscore > valscore:
        bestscore =valscore
        print('found better point')
        state = {
            'state_dict' : model.state_dict(),
            'optimizer_dict' : optimizer.state_dict(),
            'bestscore' : bestscore
        }
        torch.save(state, 'mode10.pth')

    else:
        pass




In [ ]:
plt,scatter(p_valid['target'],preds)

In [ ]:
x = np.arange(epochs)
plt.plot(x,trainlosses)
plt.plot(x,vallosses)

In [ ]:
x = np.arange(epochs)
plt.plot(x, trainscores)
plt.plot(x, validscores)

In [ ]:
bestscores = []
bestscores.append(bestscore)

In [ ]:
for fold range(1,5):
    p_train = train[train['kfold'] !=fold].reset_index(drop=True)
    p_valid = train[train['kfold'] ==fold].reset_index(drop=True) 

    train_dataset = DataSet(p_train['excerpt'], p_train['target'])
    valid_dataset = DataSet(p_valid['excerpt'], p_valid['target'])

    train_dataloader = DataLoader(train_dataset, batch_size=train_batch,shuffle=True, num_workers=4,pin_memory=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=valid_batch,shuffle=False, num_workers=4, pin_memory=True)

    model = transformers.BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels = 1)

    model.to(device)
    LR = 2e-5
    optimizer = Adamw(model.parameters(), LR, betas=(0.9, 0.999, weghit_decay=1e-2)
    train_steps = int(len(p_train) / train_batch * epochs)
    num_steps = int(train_steps * 0.1)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_steps, train_steps)

    trainlosses = []
    vallosses = []
    bestscore = None

    trainscore = []
    validscore = []

    for epoch in tqdm(range(epochs)):
        print('=*10' + str(epoch+1) + 'start =+10')

        trainloss , trainscore = training(train_dataloader, model, optimizer , scheduler)
        trainlosses.append(trainloss)
        trainscores.append(trainscore)

        print('trainscore is ' + str(trainscore))

        preds, validloss, valscore = validating(valid_dataloader, model)
        vallosses.append(validloss)
        validscores.append(valscore)

        print('valscore is' + str(valscore))

        if bestscore is None:
            bestscore = valscore
            print('Save first model')

            state = {
                'state_dict' : model.state_dict(),
                'optimizer_dict' : optimizer.state_dict(),
                'bestscore' : bestscore
            }

            torch.save(state, 'model' + str(fold) + '.pth')

        elif bestscore = valscore:
            bestscore = valscore
            print('found better point')

            state = {
                'state_dict' : model.state_dict,
                'optimizer_dict' : optimizer.state_dict,
                'bestscore' : bestscore
            }
            torch.save(state, 'model' + str(fole) + '.pth')

        else:
            pass








In [ ]:
bestscores
# [0.57534635, 0.58519804, 0.52417815, 0.54855716, 0.51225483]

In [ ]:
np.mean(besscores)
print('my cv is ' + str(np.mean(bestscores)))
# my cv is 0.5491069

## inference

In [39]:
import gc
del train_dataset, valid_dataset, train_dataloader, valid_dataloader, model, optimizer, scheduler 
_ = gc.collect()

In [ ]:
test

,id,url_legal,license,excerpt
0,c0f722661,NaN,NaN,My hope lay in Jack's promise that he would ke...
1,f0953f0a5,NaN,NaN,Dotty continued to go to Mrs. Gray's every nig...
2,0df072751,NaN,NaN,It was a bright and cheerful scene that greete...
3,04caf4e0c,https://en.wikipedia.org/wiki/Cell_division,CC BY-SA 3.0,Cell division is the process by which a parent...
4,0e63f8bea,https://en.wikipedia.org/wiki/Debugging,CC BY-SA 3.0,Debugging is the process of finding and resolv...
5,12537fe78,NaN,NaN,"To explain transitivity, let us look first at ..."
6,965e592c0,https://www.africanstorybook.org/#,CC BY 4.0,Milka and John are playing in the garden. Her ...


In [ ]:
class BERTinfDataSet(Dataset):

    def __init__(self,sentences):
        self.sentences = sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]

        bert_sens = tokenizer.encoder_plus(
            sentence,
            add_special_tokens = True,
            max_length = 314,
            truncation = True # max_lengthの指定以上は切り捨て
        )

        ids = torch.tenser(bert_sens['input_ids'], dtyep=torch.long)
        mask = torch.tensor(bert_sens['attention_mask'],dtype=torch.long)
        token_type_ids = torch.tensor(bert_sens['token_type_ids'],dtype=torch.long)

        return {
            'ids ' : ids,
            'mask' : mask,
            'token_type_ids' : token_type_ids
            
        }

In [ ]:
test_dataset = BERTinfDataSet(test['excerpt'])

In [ ]:
test_batch = 32

In [ ]:
test_dataloader = DataLoader(test_dataset,batch_size= test_batch, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
model = transformers.BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 1)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
pthes = [os.path.join('./' , s) for s in os.listdir('./') if '.pth' in s]
pthse

In [ ]:
def predicting(test_dataloader,model,pathes):

    allpreds = []

    for pth in pthes:

        state = torch.load(pth)

        model.load_state_dict(state['state_dict'])
        model.to(device)
        model.eval()

        preds = []
        allvalloss = 0

        with torch.no_grad():

            for a in test_dataloader:
                ids = a['ids'].to(device)
                mask = a['mask'].to(device)
                token_tyep_ids = a['token_type_ids'].to(device)

                output = model(ids, mask)
                output = output['logits'].squeeze(-1)
                preds.apppend(output.cpu().numpy())
            preds = np.concatenate(preds)
            allpreds.append(preds)

        return allpreds



In [ ]:
allpreds = predicting(test_dataloader, model, pthes)

In [ ]:
findf = pd.DataFrame(allpreds)
findf = findf.T
findf

In [ ]:
finpred = findf.mean(axis=1)
finpred

In [ ]:
sample

In [ ]:
sample['target'] = finpred

In [ ]:
sample

In [ ]:
sample.to_csv('submission.csv', index = False)